In [1]:
import pandas as pd

**Задание 1**

Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

**Решение.**

Retention игроков. Хотим узнать, сколько игроков возвращаются после нулевого дня. 
- Разброс данных очень большой, нет смысла анализировать весь период, за много лет. Возьмём только 2019 год, группировка по месяцам
- день 0- данные о времени регистрации


In [2]:
registration = pd.read_csv("~/shared/problem1-reg_data.csv", sep=';')  # регистрация пользователей, id + время
authorisation = pd.read_csv("~/shared/problem1-auth_data.csv", sep=';')  # входы в приложение id + время

In [3]:
registration['reg_ts'] = pd.to_datetime(registration['reg_ts'], unit='s')
authorisation['auth_ts'] = pd.to_datetime(authorisation['auth_ts'], unit='s')  # приводим время в читабельный вид

In [4]:
authorisation.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [5]:
registration.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [6]:
registration.uid.duplicated().sum()

0

In [7]:
registration_sample = registration.query('"2019-01-01 00:00:00" <= reg_ts <= "2019-12-31 23:59:59"')
authorisation_sample = authorisation.query('"2019-01-01 00:00:00" <= auth_ts <= "2019-12-31 23:59:59"')

In [20]:
def retention(registrations_data, authorisations_data, number_ret_day_array):    
    """Функция для подсчета retention. Принимает параметры- дата регистрации, авторизации и кол-во дней для расчёта"""
    full_logs = authorisations_data.merge(registrations_data, how='left', on='uid')  # собираем день регистрации и логи в одну таблицу     
    full_logs['day'] = full_logs['reg_ts'].dt.to_period('M')  # месяц регистрации
    full_logs['reg_ts'] = full_logs['reg_ts'].dt.to_period('D')  # день регистрации
    full_logs['auth_ts'] = full_logs['auth_ts'].dt.to_period('D')  # день входа в игру
    full_logs['Number_days'] = full_logs.auth_ts.astype(int) - full_logs.reg_ts.astype(int)  # кол-во дней между регистрацией и авторизацией    
    full_logs = full_logs.groupby(['day', 'Number_days'], as_index=False)\
           .uid.apply(pd.Series.nunique)  # собираем данные по месяцам  
   
    full_logs = full_logs.pivot(index='day',
                  columns='Number_days',  # собираем таблицу
                  values='uid')
    
    if (0 in number_ret_day_array) == False:                    
        number_ret_day_array.insert(0, 0)
        
    full_logs = full_logs[number_ret_day_array]
    day_total = full_logs.iloc[:, 0]  # количество пользователей в месяц
    full_logs = full_logs.divide(day_total, axis=0).fillna(0)  # делим все столбцы на столбец с кол-м пользователей в месяц
    df_style = ( full_logs\
                .style\
                .format("{:.1%}"))    
    return df_style                                     

In [21]:
retention(registration_sample, authorisation_sample, [1, 2, 3, 5, 6, 7, 14, 30, 60, 90, 120])

Number_days,0,1,2,3,5,6,7,14,30,60,90,120
day,,,,,,,,,,,,
2019-01,100.0%,2.0%,4.0%,4.8%,6.4%,6.9%,6.1%,4.9%,2.8%,1.2%,1.3%,1.3%
2019-02,100.0%,2.0%,4.3%,4.6%,6.2%,6.9%,6.1%,4.7%,2.9%,1.3%,1.4%,1.2%
2019-03,100.0%,2.0%,4.2%,4.6%,6.1%,6.8%,5.8%,4.5%,2.8%,1.3%,1.1%,1.3%
2019-04,100.0%,1.9%,3.9%,4.9%,6.0%,6.7%,5.9%,4.5%,2.8%,1.3%,1.2%,1.2%
2019-05,100.0%,2.0%,4.1%,4.7%,5.9%,6.9%,6.0%,4.4%,2.7%,1.3%,1.3%,1.2%
2019-06,100.0%,2.0%,4.3%,4.7%,6.3%,6.9%,5.8%,4.5%,2.8%,1.2%,1.3%,1.3%
2019-07,100.0%,2.0%,4.1%,4.9%,6.1%,6.8%,5.8%,4.5%,2.8%,1.2%,1.3%,1.2%
2019-08,100.0%,2.2%,4.0%,4.5%,5.8%,6.8%,5.9%,4.3%,2.8%,1.2%,1.1%,1.2%
2019-09,100.0%,1.9%,4.0%,4.5%,6.0%,6.8%,6.1%,4.4%,2.8%,1.2%,1.3%,0.1%
